<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
import numpy as np
from copy import deepcopy

In [2]:
class Literal:
    def __init__(self,name,sign=True):
        '''
        Inputs:

            name (str) The literal in type string.
              
            sign (bool) Corresponds to the sign of the literal.
              If False, the literal is negated.
              Default: True.
        '''
        self.name = name
        self.sign = sign 
        
    def __neg__(self): # Overload negation
        return(Literal(self.name, sign = not self.sign))
    
    def __repr__(self): # Printing method
        rep = self.name if self.sign else ("-" + self.name)
        return(rep)
    
    def __eq__(self, other_lit): # Define equality as independent of sign
        return(self.name == other_lit.name)
        
    def __hash__(self):
        return(hash(str(self.sign) + self.name)) 

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [3]:
def degree_heuristic(KB, symbols, model):
    """
      S = degree_heuristic(clauses, symbols, model):
          clauses: The KB with a list of clauses
          symbols: A set of the current literals (we are only interested in the literal, not the sign)
          model: A dictionary giving the truth values assigned for the model
          S: A literal object representing the unassigned 
              symbol that appears the most across all clauses (should be positive)
    """
    d={} # keep dictionary with counts
    for clause in KB:
        for lit in clause:
            liter = deepcopy(lit) # create a copy not to alter the original symbol
            liter.sign = True # change sign if necessary, since all symbols are positive
            if liter not in symbols: continue # break out of the loop
            if liter not in d:
                d[liter] = 1 # initiate count
            else:
                d[liter] +=1 # add count
    S = max(d, key=d.get)
    return(S) 


def find_unit_clause(KB, symbols, model):
    """
      U = find_unit_clause(clauses, symbols, model):
          clauses: The KB with a list of clauses
          symbols: A set of the current literals (we are only interested in the literal, not the sign)
          model: A dictionary giving the truth values assigned for the model
          U:  A unit clause (currently unassigned in model)
    """
    for clause in KB:
        if len(clause) != 1:continue # break out of this loop
        for lit in clause:
            liter = deepcopy(lit) # in order not to alter the original symbol
            liter.sign = True # change sign if necessary, since symbols are all positive
            if liter in symbols: return(liter)
    U = None
    return(U)


def DPLL_algorithm(KB, symbols, model):
    
    '''
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so.
        
        satisfiable, model = DPLL_algorithm(KB, symbols, model)
        
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    
    global all_symbols
    
    if set() in KB: 
        # backtracking to closest valid parent model here
        return(False, set()) 
    
    # If no clauses left to prove, reconstruct 
    # the solution with the final values in model.
    if not KB: 
        sat_model = dict()
        for sym in all_symbols: 
            if sym in model:
                if model[sym]:
                    sat_model[sym] = True
                else:
                    sat_model[sym] = False
            else:
                # This symbol can take any value and still result in a satisfiable sentence
                sat_model[sym] = 'Any value'
        return(True, sat_model)
    
    # pick a unit clause from the sentence, if exists
    unit_clause = find_unit_clause(KB, symbols, model) 
    
    # pick most frequent literal
    max_lit = degree_heuristic(KB, symbols, model) 
    
    # Applying unit clause and degree heuristics
    
    # Most likely, we will not start out with unit clauses, so by the time
    # we do get to them, we will have exhausted the degree heuristic.
    if unit_clause is not None:
        current_lit=unit_clause
    else:
        current_lit=max_lit
    
    # Generate a list of the rest of the literals to check, 
    # i.e. excluding the current literal
    other_lits = [lit for lit in symbols if lit!=current_lit]
    
    set1 = []
    set2 = []
    
    for clause in KB:
        # Construct the clauses without positive current literal
        # (negative literal allowed)
        if current_lit not in clause:
            set1.append(set(lit for lit in clause if current_lit != lit))
        
        # Construct the clauses without negative current literal
        # (positive literal allowed)
        if -current_lit not in clause:
            set2.append(set(lit for lit in clause if current_lit != lit))
    
    model[current_lit] = True # try assigning True
    
    # pass through the rest of the clauses, assuming current literal is True
    result, m = DPLL_algorithm(set1, other_lits, model.copy())  
    if result: return(result, m)
    
    model[current_lit] = False # try assigning False
    
    # pass through the rest of the clauses, assuming current literal is False
    result1, m1 = DPLL_algorithm(set2, other_lits, model.copy())
    if result1: return(result1, m1)
    
    else:
        return(False, set())    

def DPLL_Satisfiable(KB):
    ''' 
        Calls the DPLL_algorithm() function that implements the DPLL algorithm 
        and recursively calls itself.
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
    '''
    symbols = set(sym for clause in KB for sym in clause)   
    return(DPLL_algorithm(KB, symbols, dict()))

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

<h1>Extensions</h1>

1. Complete the degree heuristic function by filling in the placeholder in Q2.
2. Complete the pure symbol and unit clause heuristics by filling in the placeholder in Q2.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [4]:
# This section will test the correct definition of the literal class

# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [5]:
# From Exercise 7.20 of Russell & Norvig 
KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

In [6]:
# Defining the global literal that I am using in my DPLL_algorithm() function.
all_symbols = {A,B,C,D,E,F}

In [7]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [8]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model = DPLL_Satisfiable(test_KB)
assert(not t)

In [9]:
# This will test DPLL on the KB from Russell & Norvig
t, model = DPLL_Satisfiable(KB)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))   

<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics, picking the most frequently used symbol.

In [10]:
# Degree Heuristic Test
S = degree_heuristic(KB, symbols, dict())
assert(S.name == 'B')

In [11]:
# Unit Clause Test
U = find_unit_clause([{A,B,C},{-C},{A, D}], {A,B,C,D}, dict())
assert(U.name == 'C')

In [ ]:
# Pure Symbol Test
P = find_pure_symbol([{A,B,C},{A,-D},{A,D}], {A,B,C,D}, dict())
assert(P.name == 'A')

### References

I adopted the idea of extending the DPLL algorithm as presented in Russel & Norvig to clause reduction from one of the previous implementations of DPLL for CS152 class by Miranda (2018). As we assume a truth value for a literal, we reduce each clause in the sentence to a representation that assumes that truth assignment to be valid. The modified clauses are then used in the next recursive DPLL call until all clauses have been proved, or we have found that the sentence is unsatisfiable.


Miranda, V. 2018, The DPLL Algorithm. Retrieved from: https://github.com/viniciusmss/CS152-Harnessing-Artificial-Intelligence-Algorithms/blob/master/Assignment%202%20-%20The%20DPLL%20Algorithm.ipynb